# Train Custom "Claudinho" Wake Word Model

Trains an openWakeWord-compatible model using **real voice recordings**.

Bypasses the complex `train.py` pipeline — directly extracts embeddings and trains a classifier.

**Runtime: GPU** (Runtime → Change runtime type → T4 GPU)

Total time: ~15-20 minutes

## 1. Setup

In [ ]:
!pip install -q openwakeword onnxruntime numpy scipy

# Get the voice recordings
!git clone -q https://github.com/claudinhocoding/claudinho.git claudinho_repo

# Download pre-computed negative features (~2000 hrs of speech/noise/music)
!wget -q https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy
# Validation negative features (~11 hrs)
!wget -q https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/validation_set_features.npy

# Download openWakeWord embedding models
import openwakeword
openwakeword.utils.download_models()

import glob
samples = sorted(glob.glob('claudinho_repo/training/positive/claudinho/*.wav'))
print(f'\n✅ Setup complete — {len(samples)} voice samples found')

## 2. Extract Embeddings from Recordings

In [ ]:
import numpy as np
import scipy.io.wavfile
import onnxruntime as ort
from pathlib import Path
import os
import random

# Load openWakeWord's feature extraction models
oww_dir = Path(openwakeword.__file__).parent / 'resources' / 'models'
mel_session = ort.InferenceSession(str(oww_dir / 'melspectrogram.onnx'))
emb_session = ort.InferenceSession(str(oww_dir / 'embedding_model.onnx'))

def audio_to_features(audio_16k):
    """
    Convert 16kHz audio to openWakeWord embedding features.
    Returns array of shape (N, 96) where N = number of 80ms windows.
    """
    # Ensure float32 normalized to [-1, 1]
    if audio_16k.dtype == np.int16:
        audio_16k = audio_16k.astype(np.float32) / 32768.0
    
    features = []
    # Process in 80ms windows (1280 samples at 16kHz)
    window_size = 1280
    # Need 76 windows for one embedding (about 6 seconds)
    n_windows = 76
    
    # Collect mel spectrograms
    mels = []
    for start in range(0, len(audio_16k) - window_size + 1, window_size):
        chunk = audio_16k[start:start + window_size]
        mel = mel_session.run(None, {'input': chunk.reshape(1, -1)})[0]
        mels.append(mel)
    
    # Convert mel windows to embeddings (76 mels -> 1 embedding)
    if len(mels) >= n_windows:
        for i in range(0, len(mels) - n_windows + 1, 8):  # step by 8 windows
            mel_batch = np.concatenate(mels[i:i + n_windows], axis=0)
            mel_batch = mel_batch.reshape(1, n_windows, 32)  # (1, 76, 32)
            embedding = emb_session.run(None, {'input': mel_batch.astype(np.float32)})[0]
            features.append(embedding.flatten())
    
    return np.array(features) if features else np.empty((0, 96))


def augment_audio(audio, sr=16000):
    """Simple augmentations: noise, volume, time shift."""
    augmented = []
    audio_f = audio.astype(np.float32)
    
    # Original
    augmented.append(audio_f)
    
    # Add noise (various levels)
    for noise_level in [0.002, 0.005, 0.01, 0.02]:
        noisy = audio_f + np.random.randn(len(audio_f)) * noise_level * 32768
        augmented.append(noisy)
    
    # Volume changes
    for gain in [0.5, 0.7, 1.3, 1.5]:
        augmented.append(audio_f * gain)
    
    # Time shift (pad start/end)
    for shift in [1600, 3200, 4800]:  # 100ms, 200ms, 300ms
        shifted = np.concatenate([np.zeros(shift), audio_f])
        augmented.append(shifted)
        shifted_back = np.concatenate([audio_f, np.zeros(shift)])
        augmented.append(shifted_back)
    
    # Speed changes (slight)
    for rate in [0.9, 0.95, 1.05, 1.1]:
        new_len = int(len(audio_f) / rate)
        resampled = np.interp(
            np.linspace(0, len(audio_f) - 1, new_len),
            np.arange(len(audio_f)),
            audio_f
        )
        augmented.append(resampled)
    
    return augmented


print('Extracting features from recordings...')

all_positive_features = []
for wav_path in samples:
    sr, audio = scipy.io.wavfile.read(wav_path)
    if sr != 16000:
        # Resample if needed
        audio = scipy.signal.resample(audio, int(len(audio) * 16000 / sr)).astype(np.int16)
    
    # Get features from original + augmented versions
    augmented_versions = augment_audio(audio)
    for aug_audio in augmented_versions:
        feats = audio_to_features(aug_audio.astype(np.float32) / 32768.0)
        if len(feats) > 0:
            all_positive_features.append(feats)

positive_features = np.concatenate(all_positive_features, axis=0)
print(f'\n✅ Positive features: {positive_features.shape[0]} embeddings from {len(samples)} recordings')
print(f'   ({positive_features.shape[0] / len(samples):.0f}x multiplication via augmentation)')

In [ ]:
# Load pre-computed negative features
print('Loading negative features...')
negative_features_raw = np.load('openwakeword_features_ACAV100M_2000_hrs_16bit.npy')

# These are stored as int16 to save space, convert to float32
if negative_features_raw.dtype == np.int16:
    negative_features = negative_features_raw.astype(np.float32) / 256.0  # stored with 8-bit fractional
else:
    negative_features = negative_features_raw.astype(np.float32)

# Subsample negatives (we don't need all 2000 hours)
max_negatives = min(len(negative_features), positive_features.shape[0] * 50)  # 50:1 ratio
indices = np.random.choice(len(negative_features), max_negatives, replace=False)
negative_features = negative_features[indices]

print(f'✅ Negative features: {negative_features.shape[0]} embeddings')
print(f'   Ratio: {negative_features.shape[0] / positive_features.shape[0]:.0f}:1 negative:positive')

# Validation negatives
val_negative = np.load('validation_set_features.npy')
if val_negative.dtype == np.int16:
    val_negative = val_negative.astype(np.float32) / 256.0
else:
    val_negative = val_negative.astype(np.float32)
print(f'✅ Validation negatives: {val_negative.shape[0]} embeddings')

## 3. Train the Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Simple DNN classifier (matches openWakeWord architecture)
class WakeWordModel(nn.Module):
    def __init__(self, input_size=96, hidden_size=32):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers(x)

# Prepare training data
X_pos = positive_features.astype(np.float32)
X_neg = negative_features.astype(np.float32)

# Normalize features
all_X = np.concatenate([X_pos, X_neg], axis=0)
feat_mean = all_X.mean(axis=0)
feat_std = all_X.std(axis=0) + 1e-8
X_pos_norm = (X_pos - feat_mean) / feat_std
X_neg_norm = (X_neg - feat_mean) / feat_std

# Split positive into train/val
n_val = max(int(len(X_pos_norm) * 0.15), 10)
perm = np.random.permutation(len(X_pos_norm))
X_pos_val = X_pos_norm[perm[:n_val]]
X_pos_train = X_pos_norm[perm[n_val:]]

# Create training set
X_train = np.concatenate([X_pos_train, X_neg_norm])
y_train = np.concatenate([np.ones(len(X_pos_train)), np.zeros(len(X_neg_norm))])

# Shuffle
shuffle_idx = np.random.permutation(len(X_train))
X_train = X_train[shuffle_idx]
y_train = y_train[shuffle_idx]

# Validation set
val_neg_norm = (val_negative.astype(np.float32) - feat_mean) / feat_std
X_val = np.concatenate([X_pos_val, val_neg_norm[:1000]])
y_val = np.concatenate([np.ones(len(X_pos_val)), np.zeros(min(1000, len(val_neg_norm)))])

# DataLoaders
train_ds = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)

print(f'Training: {len(X_pos_train)} positive + {len(X_neg_norm)} negative = {len(X_train)} total')
print(f'Validation: {len(X_pos_val)} positive + {min(1000, len(val_neg_norm))} negative')

# Train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

model = WakeWordModel(input_size=96, hidden_size=32).to(device)

# Use weighted loss (positive class is rare)
pos_weight = torch.tensor([len(X_neg_norm) / len(X_pos_train)]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Remove sigmoid for BCEWithLogitsLoss
model.layers = nn.Sequential(
    nn.Linear(96, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

best_val_acc = 0
best_state = None

for epoch in range(50):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch).squeeze()
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    scheduler.step()
    
    # Validate
    model.eval()
    with torch.no_grad():
        val_out = model(torch.FloatTensor(X_val).to(device)).squeeze()
        val_pred = (torch.sigmoid(val_out) > 0.5).cpu().numpy()
        val_acc = (val_pred == y_val).mean()
        
        # Recall on positives
        pos_mask = y_val == 1
        recall = val_pred[pos_mask].mean() if pos_mask.sum() > 0 else 0
        
        # False positive rate
        neg_mask = y_val == 0
        fpr = val_pred[neg_mask].mean() if neg_mask.sum() > 0 else 0
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch+1:3d} | Loss: {total_loss/len(train_loader):.4f} | '
              f'Val Acc: {val_acc:.3f} | Recall: {recall:.3f} | FPR: {fpr:.4f}')

# Load best model
model.load_state_dict(best_state)
print(f'\n✅ Training complete! Best validation accuracy: {best_val_acc:.3f}')

## 4. Export to ONNX

In [ ]:
import json

# Export model to ONNX (compatible with openWakeWord)
model.eval()
model_cpu = model.cpu()

# openWakeWord expects: input shape (batch, 96), output shape (batch, 1) with sigmoid
# Add sigmoid back for inference
class ExportModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
    
    def forward(self, x):
        return torch.sigmoid(self.base(x))

export_model = ExportModel(model_cpu)
export_model.eval()

dummy_input = torch.randn(1, 96)

os.makedirs('output', exist_ok=True)
onnx_path = 'output/claudinho.onnx'

torch.onnx.export(
    export_model,
    dummy_input,
    onnx_path,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}},
    opset_version=11,
)

# Save normalization params (needed for inference)
np.savez('output/claudinho_norm.npz', mean=feat_mean, std=feat_std)

size_kb = os.path.getsize(onnx_path) / 1024
print(f'✅ Model exported: {onnx_path} ({size_kb:.1f} KB)')

# Verify it loads
test_session = ort.InferenceSession(onnx_path)
test_out = test_session.run(None, {'input': dummy_input.numpy()})
print(f'✅ ONNX verification passed (output shape: {test_out[0].shape})')

In [ ]:
# Quick test: score a few positive samples vs random noise
model_session = ort.InferenceSession(onnx_path)

print('Testing model on real samples:')
for wav_path in samples[:5]:
    sr, audio = scipy.io.wavfile.read(wav_path)
    feats = audio_to_features(audio.astype(np.float32) / 32768.0)
    if len(feats) > 0:
        feats_norm = (feats - feat_mean) / feat_std
        scores = model_session.run(None, {'input': feats_norm.astype(np.float32)})[0]
        max_score = scores.max()
        print(f'  {os.path.basename(wav_path)}: score={max_score:.3f} {"✅" if max_score > 0.5 else "❌"}')

print('\nTesting on random noise (should be low):')
for i in range(3):
    noise = np.random.randn(16000 * 3).astype(np.float32) * 0.1
    feats = audio_to_features(noise)
    if len(feats) > 0:
        feats_norm = (feats - feat_mean) / feat_std
        scores = model_session.run(None, {'input': feats_norm.astype(np.float32)})[0]
        max_score = scores.max()
        print(f'  noise_{i}: score={max_score:.3f} {"✅" if max_score < 0.3 else "⚠️"}')

In [ ]:
# Download the model files
from google.colab import files

files.download('output/claudinho.onnx')
files.download('output/claudinho_norm.npz')

print('\n🎉 Downloads started!')
print('\nNext steps:')
print('  1. scp claudinho.onnx claudinho@claudinho.local:~/claudinho/models/')
print('  2. scp claudinho_norm.npz claudinho@claudinho.local:~/claudinho/models/')
print('  3. ssh claudinho@claudinho.local')
print('  4. cd ~/claudinho && git pull && sudo systemctl restart claudinho')